In [1]:
import numpy as np
import tensorflow as tf
wordsList = np.load('/home/gopal/Pictures/Hindi_sentiment_analysis/hindi_wordlist.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
#wordsList = [word.encode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('/home/gopal/Pictures/Hindi_sentiment_analysis/hindi_wordvectors.npy')
print ('Loaded the word vectors!')
wordVectors.shape

Loaded the word list!
Loaded the word vectors!


(116526, 100)

In [2]:

wordsList.index('कोसी')

25

In [3]:

from os import listdir
from os.path import isfile, join
pos = []
neg = []
neu = []
with open("/home/gopal/Pictures/Hindi_sentiment_analysis/pos_text.txt", "r") as positive:
    for line in positive:
        pos.append(line.strip('\n'))
        
with open("/home/gopal/Pictures/Hindi_sentiment_analysis/neg_txt.txt", "r") as negative:
    for line in negative:
        neg.append(line.strip('\n'))
        
with open("/home/gopal/Pictures/Hindi_sentiment_analysis/neu_text.txt", "r") as neutral:
    for line in neutral:
        neu.append(line.strip('\n'))
        
print(len(pos))
print(len(neg))
print(len(neu))
#print(pos)

2290
712
2226


In [4]:
numWords = []
for pf in pos:
#with open(pos, "r", encoding='utf-8') as f:
    #line=f.readline()
    counter = len(pf.split())
    numWords.append(counter)       
print('Positive files finished')

for nf in neg:
#with open(nf, "r", encoding='utf-8') as f:
    #line=f.read()
    counter = len(nf.split())
    numWords.append(counter)  
print('Negative files finished')


for neuf in neu:
#with open(nf, "r", encoding='utf-8') as f:
    #line=f.read()
    counter = len(neuf.split())
    numWords.append(counter)  
print('Negative files finished')

numWords[4629]

Positive files finished
Negative files finished
Negative files finished


78

In [5]:
print("numWords : ",max(numWords), numWords.index(max(numWords)))

numFiles = len(numWords)
print('The total number of files is', numFiles)
print('The total number of words in the files is', sum(numWords))
print('The average number of words in the files is', sum(numWords)/len(numWords))


numWords :  78 4629
The total number of files is 5228
The total number of words in the files is 95499
The average number of words in the files is 18.2668324407039


In [6]:
maxSeqLength = 100
numDimensions = 100 #Dimensions for each word vector

In [7]:
# # Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
# import re
# strip_special_chars = re.compile("[^A-Za-z0-9 ]+")
# https = re.compile(r'^https?:\/\/.*[\r\n]*')
# #r'http\S+',

# def cleanSentences(string):
#     string = string.lower().replace("<br />", " ")
#     string = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*',"",string, flags=re.MULTILINE)
#     #string = re.sub(r"[\'’“”?]", "", string, flags=re.I)
#     string = re.sub(r"[\“”?]", "", string, flags=re.I)
#     string = re.sub(strip_special_chars, " ", string)
#     #string = re.sub(' +', ' ', string)
#     return string


In [8]:
ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
print(ids.shape)
fileCounter = 0
for pf in pos:
    indexCounter = 0
   # cleanedLine = cleanSentences(pf)
    split = pf.split()
    for word in split:
        try:
            #print(fileCounter,indexCounter)
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 116525 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1
print("positive file done")

for nf in neg:
    indexCounter = 0
    #cleanedLine = cleanSentences(nf)
    split = nf.split()
    for word in split:
        try:
            #print(indexCounter)
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 116525 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1
print("negative file done")

        
for neuf in neu:
    indexCounter = 0
    #cleanedLine = cleanSentences(neuf)
    split = neuf.split()
    for word in split:
        try:
            #print(fileCounter,indexCounter)
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 116525 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1
print("neutral file done")

#Pass into embedding function and see if it evaluates. 



(5228, 100)
positive file done
negative file done
neutral file done


In [9]:
print(wordsList.index('लेने'))

0


In [10]:
np.save('/home/gopal/Pictures/Hindi_sentiment_analysis/idsMatrix.npy', ids)
ids = np.load('/home/gopal/Pictures/Hindi_sentiment_analysis/idsMatrix.npy')
ids[134]

array([  7142,   6167,     27,    437, 116525, 116525,      1,    137,
       116525,     52,   3833,   5393, 116525,     27, 116525,  29030,
         9322,    693, 116525,    313, 116525,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0], dtype=int32)

In [11]:
from random import randint

def getTrainBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1,5228)#36562,28867,31165
        #print(num)
        if (num < 2290): #36101
            labels.append([1,0,0])
        elif (num >= 2290 and num <= 3002): #64355
            labels.append([0,1,0])
        #elif (num > 65429 and num <= 96594): #95074
        else:
            labels.append([0,0,1])
        arr[i] = ids[num-1:num]
    return arr, labels

"""def getTestBatch():
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1,95074)
        if (num <= 36101): #36101
            labels.append([1,0,0])
        elif (num > 36101 and num <= 64355): #64355
            labels.append([0,1,0])
        elif (num > 64355 and num <= 95074):
            labels.append([0,0,1])
        arr[i] = ids[num-1:num]
    return arr, labels"""


'def getTestBatch():\n    labels = []\n    arr = np.zeros([batchSize, maxSeqLength])\n    for i in range(batchSize):\n        num = randint(1,95074)\n        if (num <= 36101): #36101\n            labels.append([1,0,0])\n        elif (num > 36101 and num <= 64355): #64355\n            labels.append([0,1,0])\n        elif (num > 64355 and num <= 95074):\n            labels.append([0,0,1])\n        arr[i] = ids[num-1:num]\n    return arr, labels'

In [12]:
batchSize = 512 #(500,256 gives good acc)
lstmUnits = 256 #64(32-batch,note worked),size
numClasses = 3
iterations =100000

import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
#prediction = (tf.matmul(last, weight) + bias)
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

'''loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)'''


loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
beta = 0.01

# Loss function using L2 Regularization
regularizer = tf.nn.l2_loss(weight)
loss = tf.reduce_mean(loss + beta * regularizer)




Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [13]:
optimizer = tf.train.AdamOptimizer().minimize(loss)


Instructions for updating:
Use tf.cast instead.


In [ ]:
import datetime
with tf.Session() as sess:
    tf.summary.scalar('Loss', loss)
    tf.summary.scalar('Accuracy', accuracy)
    merged = tf.summary.merge_all()
    logdir = "/home/gopal/Pictures/Hindi_sentiment_analysis/tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
    writer = tf.summary.FileWriter(logdir, sess.graph)

#it gives us the details 
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
config.log_device_placement=True
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess = tf.Session(config=config)

#sess = tf.InteractiveSession()

saver = tf.train.Saver()

sess.run(tf.global_variables_initializer())


for i in range(iterations):
    #Next Batch of datasets
    print(i)
    nextBatch, nextBatchLabels = getTrainBatch();
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
   
    #Write summary to Tensorboard
    if (i % 50 == 0):
        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
        writer.add_summary(summary, i)
        
    #Save the network every 10,000 training iterations
    if (i % 10000 == 0 and i != 0):
        save_path = saver.save(sess, "/home/gopal/Pictures/Hindi_sentiment_analysis/hindi/hindi.ckpt", global_step=i)
        print("saved to %s" % save_path)
writer.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
